In [1]:
import pandas as pd
from exosyspop.populations import BinaryPopulation
from exosyspop.populations import TRILEGAL_BGBinaryPopulation
from exosyspop.populations import KeplerBinaryPopulation, PoissonPlanetPopulation

targets = pd.read_hdf('targets.h5')
bgstars = pd.read_hdf('bgstars.h5')

# Sanitize dtypes of targets DataFrame
for c in targets.columns:
    if targets[c].dtype == object:
        targets.loc[:,c] = targets.loc[:,c].astype(str)


import logging
rootLogger = logging.getLogger()

In [2]:
ebpop = KeplerBinaryPopulation(targets)
bgpop = TRILEGAL_BGBinaryPopulation(targets, bgstars)
plpop = PoissonPlanetPopulation(targets)

In [3]:
import os, os.path, shutil

class PopulationMixture(object):
    def __init__(self, poplist):
        self.poplist = poplist
        
    @property
    def param_names(self):
        p = []
        for pop in self.poplist:
            p += pop.param_names
        return list(set(p))
    
    def set_params(self, **kwargs):
        for pop in self.poplist:
            pop.set_params(**kwargs)
                        
    def train_trap(self, **kwargs):
        return [p._train_trap(**kwargs) for p in self.poplist]
                
    def observe(self, **kwargs):
        obs = []
        for pop in self.poplist:
            o = pop.observe(new=True, **kwargs)
            o.loc[:, 'population'] = pop.name
            obs.append(o)
            
        return pd.concat(obs)

    def save(self, folder, overwrite=False):
        if os.path.exists(folder):
            if overwrite:
                shutil.rmtree(folder)
            else:
                raise IOError('{} exists.  Set overwrite=True to overwrite.'.format(folder))
        os.makedirs(folder)
        for pop in self.poplist:
            pop.save(os.path.join(folder,pop.name))
        
        
    @classmethod
    def load(cls, folder):
        names = os.listdir(folder)
        poplist = []
        for name in names:
            f = os.path.join(folder,name)
            poplist.append(BinaryPopulation.load(f))
        return cls(poplist)
    
pops = PopulationMixture([ebpop, bgpop, plpop])

In [4]:
plines = pops.train_trap(N=3000)

EB: dmag regressor trained, R2=0.999558155833
EB: qR regressor trained, R2=0.999546313676
EB: Depth trained: R2=0.998614562544
EB: Duration trained: R2=0.998929784826
EB: Slope trained: R2=0.990006662911
BGEB: dmag regressor trained, R2=0.964499765506
BGEB: qR regressor trained, R2=0.924320342896
BGEB: Depth trained: R2=0.994435573481
BGEB: Duration trained: R2=0.998927366196
BGEB: Slope trained: R2=0.986467437686
Planet: Depth trained: R2=0.999057963191
Planet: Duration trained: R2=0.972555972581
Planet: Slope trained: R2=0.983781609138


In [5]:
pops.save('mixture', overwrite=True)

In [6]:
pops2 = PopulationMixture.load('mixture')

In [7]:
pops2.poplist

In [8]:
obs = pops2.observe()
len(obs)

4683

In [9]:
%prun pops.observe(regr_trap=True)

In [10]:
obs.groupby('population').count()

,T14_pri,T14_sec,T23_pri,T23_sec,a,aR,b_pri,b_sec,b_target,d_pri,...,inc,k,n_pri,n_sec,occ,period,phase_sec,target_mag,tra,w
population,,,,,,,,,,,,,,,,,,,,,
BGEB,475,475,475,475,475,475,475,475,475,475,...,475,475,475,475,475,475,475,475,475,475
EB,523,523,523,523,523,523,523,523,523,523,...,523,523,523,523,523,523,523,0,523,523
Planet,3685,3685,3685,3685,3685,3685,3685,3685,3685,3685,...,3685,3685,3685,3685,3685,3685,3685,0,3685,3685
